# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9a34579e80>
├── 'a' --> tensor([[ 0.2312,  0.2358, -1.8397],
│                   [ 0.2122, -0.4542, -1.0658]])
└── 'x' --> <FastTreeValue 0x7f9a34579f70>
    └── 'c' --> tensor([[ 0.7214, -0.1884,  1.3060, -0.3688],
                        [ 2.3987, -1.1065, -0.6570,  0.1796],
                        [ 0.3553, -0.7181,  1.2098, -1.3395]])

In [4]:
t.a

tensor([[ 0.2312,  0.2358, -1.8397],
        [ 0.2122, -0.4542, -1.0658]])

In [5]:
%timeit t.a

63.4 ns ± 0.0706 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9a34579e80>
├── 'a' --> tensor([[-0.6588, -0.1879, -1.5126],
│                   [-0.3961,  0.1859,  0.3920]])
└── 'x' --> <FastTreeValue 0x7f9a34579f70>
    └── 'c' --> tensor([[ 0.7214, -0.1884,  1.3060, -0.3688],
                        [ 2.3987, -1.1065, -0.6570,  0.1796],
                        [ 0.3553, -0.7181,  1.2098, -1.3395]])

In [7]:
%timeit t.a = new_value

63.3 ns ± 0.0189 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2312,  0.2358, -1.8397],
               [ 0.2122, -0.4542, -1.0658]]),
    x: Batch(
           c: tensor([[ 0.7214, -0.1884,  1.3060, -0.3688],
                      [ 2.3987, -1.1065, -0.6570,  0.1796],
                      [ 0.3553, -0.7181,  1.2098, -1.3395]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2312,  0.2358, -1.8397],
        [ 0.2122, -0.4542, -1.0658]])

In [11]:
%timeit b.a

59.9 ns ± 0.0319 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3703, -0.5983,  1.5281],
               [ 0.2532, -0.1294,  0.3268]]),
    x: Batch(
           c: tensor([[ 0.7214, -0.1884,  1.3060, -0.3688],
                      [ 2.3987, -1.1065, -0.6570,  0.1796],
                      [ 0.3553, -0.7181,  1.2098, -1.3395]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.344 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

839 ns ± 3.34 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 21.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 576 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f998fe95fd0>
├── 'a' --> tensor([[[ 0.2312,  0.2358, -1.8397],
│                    [ 0.2122, -0.4542, -1.0658]],
│           
│                   [[ 0.2312,  0.2358, -1.8397],
│                    [ 0.2122, -0.4542, -1.0658]],
│           
│                   [[ 0.2312,  0.2358, -1.8397],
│                    [ 0.2122, -0.4542, -1.0658]],
│           
│                   [[ 0.2312,  0.2358, -1.8397],
│                    [ 0.2122, -0.4542, -1.0658]],
│           
│                   [[ 0.2312,  0.2358, -1.8397],
│                    [ 0.2122, -0.4542, -1.0658]],
│           
│                   [[ 0.2312,  0.2358, -1.8397],
│                    [ 0.2122, -0.4542, -1.0658]],
│           
│                   [[ 0.2312,  0.2358, -1.8397],
│                    [ 0.2122, -0.4542, -1.0658]],
│           
│                   [[ 0.2312,  0.2358, -1.8397],
│                    [ 0.2122, -0.4542, -1.0658]]])
└── 'x' --> <FastTreeValue 0x7f99891e6f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 49.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f99891e5f40>
├── 'a' --> tensor([[ 0.2312,  0.2358, -1.8397],
│                   [ 0.2122, -0.4542, -1.0658],
│                   [ 0.2312,  0.2358, -1.8397],
│                   [ 0.2122, -0.4542, -1.0658],
│                   [ 0.2312,  0.2358, -1.8397],
│                   [ 0.2122, -0.4542, -1.0658],
│                   [ 0.2312,  0.2358, -1.8397],
│                   [ 0.2122, -0.4542, -1.0658],
│                   [ 0.2312,  0.2358, -1.8397],
│                   [ 0.2122, -0.4542, -1.0658],
│                   [ 0.2312,  0.2358, -1.8397],
│                   [ 0.2122, -0.4542, -1.0658],
│                   [ 0.2312,  0.2358, -1.8397],
│                   [ 0.2122, -0.4542, -1.0658],
│                   [ 0.2312,  0.2358, -1.8397],
│                   [ 0.2122, -0.4542, -1.0658]])
└── 'x' --> <FastTreeValue 0x7f99898e6a00>
    └── 'c' --> tensor([[ 0.7214, -0.1884,  1.3060, -0.3688],
                        [ 2.3987, -1.1065, -0.6570,  0.1796],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 34.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.3 µs ± 52.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7214, -0.1884,  1.3060, -0.3688],
                       [ 2.3987, -1.1065, -0.6570,  0.1796],
                       [ 0.3553, -0.7181,  1.2098, -1.3395]],
              
                      [[ 0.7214, -0.1884,  1.3060, -0.3688],
                       [ 2.3987, -1.1065, -0.6570,  0.1796],
                       [ 0.3553, -0.7181,  1.2098, -1.3395]],
              
                      [[ 0.7214, -0.1884,  1.3060, -0.3688],
                       [ 2.3987, -1.1065, -0.6570,  0.1796],
                       [ 0.3553, -0.7181,  1.2098, -1.3395]],
              
                      [[ 0.7214, -0.1884,  1.3060, -0.3688],
                       [ 2.3987, -1.1065, -0.6570,  0.1796],
                       [ 0.3553, -0.7181,  1.2098, -1.3395]],
              
                      [[ 0.7214, -0.1884,  1.3060, -0.3688],
                       [ 2.3987, -1.1065, -0.6570,  0.1796],
                       [ 0.3553, -0.7181,  1.2098, -1.3395]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 186 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7214, -0.1884,  1.3060, -0.3688],
                      [ 2.3987, -1.1065, -0.6570,  0.1796],
                      [ 0.3553, -0.7181,  1.2098, -1.3395],
                      [ 0.7214, -0.1884,  1.3060, -0.3688],
                      [ 2.3987, -1.1065, -0.6570,  0.1796],
                      [ 0.3553, -0.7181,  1.2098, -1.3395],
                      [ 0.7214, -0.1884,  1.3060, -0.3688],
                      [ 2.3987, -1.1065, -0.6570,  0.1796],
                      [ 0.3553, -0.7181,  1.2098, -1.3395],
                      [ 0.7214, -0.1884,  1.3060, -0.3688],
                      [ 2.3987, -1.1065, -0.6570,  0.1796],
                      [ 0.3553, -0.7181,  1.2098, -1.3395],
                      [ 0.7214, -0.1884,  1.3060, -0.3688],
                      [ 2.3987, -1.1065, -0.6570,  0.1796],
                      [ 0.3553, -0.7181,  1.2098, -1.3395],
                      [ 0.7214, -0.1884,  1.3060, -0.3688],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 404 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
